### Importing Packages

In [1]:
import sys
sys.path.insert(0, 'C:/Users/enioh/Documents/Github/MMA-ML-Model')

from os.path import abspath, dirname, join
from src.models.train_test import train_test_split, X_y_split

import pandas as pd
import numpy as np

### Importing Fully Processed Data

In [2]:
df = pd.read_csv(abspath(join(dirname(abspath('')), 'data/', 'processed/', 'extracted_stats.csv')))

### Splitting Dataset into training and test sets

## MODEL 1: XGBoost 

In [3]:
from src.models.XGBoost_Model import make_model
from src.models.train_model import train_model
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

clf = make_model()
clf, split_data = train_model(clf, df)

X_train, y_train, X_test, y_test = split_data

C:\Users/enioh/Documents/Github/MMA-ML-Model\src\models\train_model.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  post_comp_cols = df.loc[:, ~df.columns.str.contains('precomp_([a-zA-Z_]+)_vs_opp', regex=True)].columns.to_list() # All columns that do not have the precomp_ id or vs_opp id


Model Training Accuracy: 0.7818480043739748
Model Test Accuracy: 0.5840978593272171


In [4]:
X_train.shape

(7316, 1472)

### Not that good, lets optimize using Optuna

In [ ]:
from src.models.Optuna import OptunaTuning

tuner = OptunaTuning(X_test, y_test, X_train, y_train)
tuner.run()

### Optimal Model

In [13]:
import xgboost as xgb

bst = xgb.XGBClassifier(reg_lambda = 0.21262699261144707, 
                        alpha = 0.002190735457731732, 
                        tree_method = 'gpu_hist',
                        objective = 'binary:logistic',
                        verbosity = 0,
                        n_jobs = -1, 
                        learning_rate = 0.01830371431723197,
                        min_child_weight = 12, 
                        max_depth = 6, 
                        max_delta_step = 5, 
                        subsample = 0.12516270393991097,
                        colsample_bytree = 0.39799515236683536,
                        gamma = 0.225275077908943, 
                        n_estimators = 315,
                        eta = 0.11452245768637671)

bst.fit(X_train, y_train)
y_pred_train = bst.predict(X_train)
y_pred_test = bst.predict(X_test)
train_score = accuracy_score(y_train, y_pred_train)
test_score = accuracy_score(y_test, y_pred_test)
print(train_score)
print(test_score)

0.6880009292600766
0.5779816513761468
